### Домашнее задание <a class="anchor" id="hw"></a><center>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

__Задача:__ подойдет ли репетитор для подготовки к экзамену по математике

In [2]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [3]:
def standard_scale(X):
    mean = X.mean(axis=0)
    std = X.std(axis=0)
    return (X - mean) / std

In [4]:
X_st = X.copy().astype(np.float64)
X_st[:, 1:4] = standard_scale(X_st[:, 1:4])
X_st

array([[ 1.        , -0.70710678, -0.97958969, -0.89625816],
       [ 1.        , -0.70710678, -0.56713087, -0.89625816],
       [ 1.        , -0.35355339, -0.46401617,  0.38411064],
       [ 1.        ,  0.70710678, -0.77336028, -0.89625816],
       [ 1.        ,  0.        ,  0.97958969,  0.38411064],
       [ 1.        , -1.06066017, -0.36090146, -0.89625816],
       [ 1.        ,  0.70710678,  1.08270439,  1.66447944],
       [ 1.        ,  2.47487373,  2.11385144,  1.66447944],
       [ 1.        , -0.70710678, -1.08270439, -0.89625816],
       [ 1.        , -0.35355339,  0.05155735,  0.38411064]])

**1. *Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.**

1 метод

In [5]:
def calc_logloss(y, y_pred):
    y_pred = np.where(y_pred == 1, 1 - 1e-8, y_pred)
    y_pred = np.where(y_pred == 0, 0 + 1e-8, y_pred)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [6]:
eps = 1e-8

np.clip([1, 0.5, 0], a_min=eps, a_max=1 - eps)

array([9.9999999e-01, 5.0000000e-01, 1.0000000e-08])

In [7]:
def calc_logloss(y, y_pred):
    eps = 1e-8
    y_pred = np.clip(y_pred, eps, 1 - eps)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

2 метод

In [8]:
def calc_logloss(y, y_pred):
    err = 0
    for i in range(len(y)):
        if y_pred[i] == 0:
            err += (1.0 - y[i]) * np.log(1.0 - y_pred[i])
        elif y_pred[i] == 1:
            err += y[i] * np.log(y_pred[i])
        else:
            err += y[i] * np.log(y_pred[i]) + (1.0 - y[i]) * np.log(1.0 - y_pred[i])
    return - err / len(y)

In [9]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0])
calc_logloss(y1, y_pred1)

0.05268025782891314

In [10]:
y1 = np.array([1, 0])
y_pred1 = np.array([1, 0.1])
calc_logloss(y1, y_pred1)

0.05268025782891314

In [11]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss(y1, y_pred1)

0.10536051565782628

**2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.**

In [12]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [13]:
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    
    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        
        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
            
    final_error = calc_logloss(y, y_pred)
    return W, final_error

1 метод

аналог GridSearch

In [14]:
from tqdm import tqdm

In [15]:
np.logspace(2, 4, 4, dtype=np.int), np.linspace(1e-2, 5, 10)

(array([  100,   464,  2154, 10000]),
 array([0.01      , 0.56444444, 1.11888889, 1.67333333, 2.22777778,
        2.78222222, 3.33666667, 3.89111111, 4.44555556, 5.        ]))

In [16]:
iterations = np.logspace(2, 4, 4, dtype=np.int)
etas = np.linspace(1e-2, 5, 10)

best_error = np.inf
best_params = {}

for iteration in tqdm(iterations):
    for eta in etas:
        W, error = eval_model(X_st, y, iterations=iteration, eta=eta)
        if error < best_error:
            best_error = error
            best_params = {
                'iteration': iteration,
                'eta': eta
            }

print(f'Ошибка {best_error} при параметрах {best_params}')

  0%|          | 0/4 [00:00<?, ?it/s]

0 [ 0.49635916 -0.1409725   0.64535427  1.52178506] 0.760958797591889
10 [ 0.49265881 -0.16787071  0.62221328  1.50954192] 0.7467006818024327
20 [ 0.48868992 -0.19442917  0.59944411  1.49767566] 0.7328651153682048
30 [ 0.48446029 -0.22063955  0.57705409  1.48619252] 0.7194524935754576
40 [ 0.47997869 -0.24649338  0.55505065  1.4750986 ] 0.7064630339679188
50 [ 0.47525478 -0.27198192  0.53344129  1.46439984] 0.6938967869573868
60 [ 0.47029912 -0.29709626  0.51223358  1.45410204] 0.6817536325961876
70 [ 0.46512312 -0.32182725  0.49143512  1.44421083] 0.6700332616838688
80 [ 0.45973898 -0.34616556  0.47105359  1.4347317 ] 0.658735139989009
90 [ 0.45415964 -0.37010173  0.4510966   1.42566993] 0.6478584551577353
0 [ 0.47667685 -0.29112687  0.51593213  1.45276794] 0.760958797591889
10 [ 0.15302827 -1.07349673 -0.05330995  1.447242  ] 0.458878668542744
20 [ 0.0338664  -1.33389885 -0.1641304   1.79467485] 0.41846490870843844
30 [-2.07371606e-03 -1.53135455e+00 -2.56777284e-01  2.10047940e+00] 

 25%|██▌       | 1/4 [00:00<00:01,  2.87it/s]

[-0.00618543 -2.0314492  -0.54660905  2.9407954 ] 0.3452925966686693
20 [ 0.06511937 -2.47646445 -0.95262246  3.88656217] 0.3043309289424501
30 [ 0.15385974 -2.76171976 -1.28483471  4.5779519 ] 0.2831549234360089
40 [ 0.24344915 -2.97992487 -1.5677191   5.14649003] 0.2690307681868176
50 [ 0.32969263 -3.16454021 -1.81664428  5.64274134] 0.25835000732224234
60 [ 0.41168091 -3.32988231 -2.04088394  6.09125239] 0.24967101192571062
70 [ 0.48948607 -3.48285308 -2.2464415   6.50558411] 0.24229411414025157
80 [ 0.56345441 -3.62708493 -2.43739097  6.89392971] 0.23583399933176627
90 [ 0.63397518 -3.76463807 -2.61659116  7.2615806 ] 0.23005861671400737
0 [ 0.35858295 -1.19205313 -0.26060073  1.03866522] 0.760958797591889
10 [ 1.23480680e-03 -2.23281658e+00 -6.56061737e-01  3.17783014e+00] 0.33408055978444406
20 [ 0.09769048 -2.62092803 -1.08165749  4.19032726] 0.2947503263118352
30 [ 0.20422929 -2.90039123 -1.44600153  4.9168339 ] 0.27470037894068977
40 [ 0.30718189 -3.12307208 -1.75216395  5.520

 50%|█████     | 2/4 [00:01<00:01,  1.84it/s]

0 [ 0.33890064 -1.3422075  -0.39002287  0.96964809] 0.760958797591889
0 [ 0.31921832 -1.49236188 -0.51944502  0.90063097] 0.760958797591889
0 [ 0.49635916 -0.1409725   0.64535427  1.52178506] 0.760958797591889
0 [ 0.47667685 -0.29112687  0.51593213  1.45276794] 0.760958797591889
0 [ 0.45699453 -0.44128125  0.38650998  1.38375082] 0.760958797591889
0 [ 0.43731221 -0.59143562  0.25708784  1.3147337 ] 0.760958797591889
0 [ 0.4176299  -0.74159     0.1276657   1.24571658] 0.760958797591889
0 [ 0.39794758 -0.89174437 -0.00175644  1.17669946] 0.760958797591889
0 [ 0.37826527 -1.04189875 -0.13117859  1.10768234] 0.760958797591889
0 [ 0.35858295 -1.19205313 -0.26060073  1.03866522] 0.760958797591889


 75%|███████▌  | 3/4 [00:04<00:01,  1.70s/it]

0 [ 0.33890064 -1.3422075  -0.39002287  0.96964809] 0.760958797591889
0 [ 0.31921832 -1.49236188 -0.51944502  0.90063097] 0.760958797591889
0 [ 0.49635916 -0.1409725   0.64535427  1.52178506] 0.760958797591889
1000 [ 0.06528341 -1.25315687 -0.13031431  1.68209805] 0.42642421911106754
2000 [-0.00591593 -1.60120112 -0.2960273   2.21707777] 0.38196097010869967
3000 [-0.01097467 -1.85119187 -0.44498757  2.64578105] 0.3549927868836243
4000 [-2.64430856e-04 -2.04430387e+00 -5.82222091e-01  3.00431189e+00] 0.33646376256331906
5000 [ 0.01719932 -2.20052369 -0.70941905  3.31414696] 0.32274977674382266
6000 [ 0.03850056 -2.33129977 -0.82799326  3.58846186] 0.3120492881925758
7000 [ 0.06216276 -2.44374222 -0.93914508  3.83587188] 0.3033639688244948
8000 [ 0.08727955 -2.54253835 -1.04387339  4.06228822] 0.2960963045725309
9000 [ 0.11325547 -2.63091817 -1.14300269  4.27192266] 0.28986691616198357
0 [ 0.47667685 -0.29112687  0.51593213  1.45276794] 0.760958797591889
1000 [ 1.09439377 -4.68780498 -3.

100%|██████████| 4/4 [00:14<00:00,  3.61s/it]

8000 [  9.46116677 -23.0072556  -30.00588618  63.19706309] 0.04023701781384288
9000 [  9.96965864 -24.01983811 -31.78454258  66.61013199] 0.03701577777302838
Ошибка 0.03424062515544959 при параметрах {'iteration': 10000, 'eta': 5.0}


2 метод

Вручную

In [17]:
W, error = eval_model(X_st, y, iterations=100, eta=1)

0 [ 0.46121499 -0.40908382  0.41426183  1.39855008] 0.760958797591889
10 [ 0.0387634  -1.30386446 -0.15001914  1.74758184] 0.4254600283700163
20 [-0.01161527 -1.64023671 -0.31196552  2.27533881] 0.38108858271134755
30 [-0.01156172 -1.88360488 -0.46035705  2.69723081] 0.3542235298084087
40 [ 9.33444488e-04 -2.07189803e+00 -5.97280973e-01  3.05064764e+00] 0.3358002141655348
50 [ 0.01934395 -2.22448855 -0.72422385  3.35653644] 0.3221745410588311
60 [ 0.04127542 -2.35245169 -0.84255667  3.6277315 ] 0.3115442516500474
70 [ 0.06539266 -2.46266676 -0.95347146  3.87262358] 0.3029143996576552
80 [ 0.090846   -2.55967079 -1.05796576  4.09697143] 0.2956909276718162
90 [ 0.11707191 -2.64659092 -1.15686395  4.30488038] 0.28949721467181433


**3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).**

In [18]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(X, W))
    return y_pred_proba

In [19]:
calc_pred_proba(W, X_st)

array([0.32380055, 0.22296142, 0.96767742, 0.00787362, 0.65604029,
       0.36735518, 0.98703363, 0.14768606, 0.35244079, 0.94041789])

**4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).**

In [20]:
def calc_pred(W, X):
    y_pred_proba = calc_pred_proba(W, X)
    y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    return y_pred

In [21]:
pred = calc_pred(W, X_st)
pred

array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1])

**5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.**

*Accuracy*

In [22]:
def accuracy(y, y_pred):
    accuracy = np.mean(y == y_pred)
    return accuracy

In [23]:
print(f'True \t  {y}')
print(f'Predicted {pred}')

True 	  [0 0 1 0 1 0 1 0 1 1]
Predicted [0 0 1 0 1 0 1 0 0 1]


In [24]:
accuracy(y, pred)

0.9

*Confusion Matrix*

In [25]:
# TP  FP
# FN  TN

In [26]:
def confusion_matrix(y, y_pred):
    cm = np.zeros((2, 2))
    for i in range(len(y)): 
        if y[i] == y_pred[i] == 1: # TP
            cm[0][0] += 1
        elif y[i] == y_pred[i] == 0: # TN
            cm[1][1] += 1
        elif y[i] != y_pred[i] and y[i] == 1: # FN
            cm[1][0] += 1
        elif y[i] != y_pred[i] and y[i] == 0: # FP
            cm[0][1] += 1
    return cm

In [27]:
cm = confusion_matrix(y, pred)
cm

array([[4., 0.],
       [1., 5.]])

*Precision*

In [28]:
def precision(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]
    precision = TP / (TP + FP)
    return precision

In [29]:
precision(y, pred)

1.0

*Recall*

In [30]:
def recall(y, y_pred):
    cm = confusion_matrix(y, y_pred)
    TP = cm[0][0]
    TN = cm[1][1]
    FP = cm[0][1]
    FN = cm[1][0]
    recall = TP / (TP + FN)
    return recall

In [31]:
recall(y, pred)

0.8

*F-score*

In [32]:
def f_score(y, y_pred):
    pr = precision(y, y_pred)
    rec = recall(y, y_pred)
    f_score = 2 * pr * rec / (pr + rec)
    return f_score

In [33]:
f_score(y, pred)

0.888888888888889

In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


print(f'accuracy {accuracy_score(y, pred)}')
print(f'precision {precision_score(y, pred)}')
print(f'recall {recall_score(y, pred)}')
print(f'f-score {f1_score(y, pred)}')
print(f'\n {confusion_matrix(y, pred)}')

accuracy 0.9
precision 1.0
recall 0.8
f-score 0.888888888888889

 [[5 0]
 [1 4]]
